# Step 3 : generate $Q_\Delta$ coefficients

📜 _We denote by_ $Q_\Delta$ _the approximation of a given_ $Q$ _matrix related to any_ $Q$_-coefficients (from collocation, RK, ...) :_

$$
\begin{array}
    {c|c}
    \tau & Q \\
    \hline
    & w^\top
\end{array}
$$

There is **two approaches** to generate $Q_\Delta$ approximations, from which you can choose in function of your needs and preferences.

## Use a generic function

The quick easy way, simply import :

In [1]:
from qmat import genQDeltaCoeffs

Then considering, e.g, the following collocation method :

In [2]:
from qmat.qcoeff.collocation import Collocation
coll = Collocation.getInstance()  # use default parameters : 4 LEGENDRE RADAU-RIGHT nodes

we can generate the $Q_\Delta$ matrix from a Backward Euler discretization between the nodes (as in the original SDC method) :

In [3]:
QDelta = genQDeltaCoeffs("BE", nodes=coll.nodes)
print(QDelta)

[[0.08858796 0.         0.         0.        ]
 [0.08858796 0.3208789  0.         0.        ]
 [0.08858796 0.3208789  0.3781926  0.        ]
 [0.08858796 0.3208789  0.3781926  0.21234054]]


... or get the LU approximation from [Weiser 2015](https://doi.org/10.1007/s10543-014-0540-y) :

In [4]:
QDelta = genQDeltaCoeffs("LU", Q=coll.Q)
print(QDelta)

[[0.11299948 0.         0.         0.        ]
 [0.234384   0.29050213 0.         0.        ]
 [0.21668178 0.48341808 0.30825766 0.        ]
 [0.22046221 0.46683684 0.44141588 0.11764706]]


... or the diagonal approximation from [van der Houwen & Sommeijer 1991](https://doi.org/10.1137/0912054) :

In [5]:
QDelta = genQDeltaCoeffs("VDHS", nNodes=coll.nNodes, nodeType=coll.nodeType, quadType=coll.quadType)
print(QDelta)

[[0.32049937 0.         0.         0.        ]
 [0.         0.08915379 0.         0.        ]
 [0.         0.         0.18173956 0.        ]
 [0.         0.         0.         0.2333628 ]]


... or even some magical diagonal coefficients obtained with a black box optimizer that does not exists anymore from [Speck 2021](https://zenodo.org/records/5775971): 

In [6]:
QDelta = genQDeltaCoeffs("MIN3", nNodes=coll.nNodes, nodeType=coll.nodeType, quadType=coll.quadType)
print(QDelta)

[[0.31987868 0.         0.         0.        ]
 [0.         0.08887606 0.         0.        ]
 [0.         0.         0.18123663 0.        ]
 [0.         0.         0.         0.23273925]]


> 🔍 ... yes, the `MIN3` coefficients are almost the same as those from `VDHS`, even if both where obtained independently ! 
> But who really _never ever_ re-invented the wheel in research 😉

Note that depending on the requested approximation, different arguments may be required for the $Q_\Delta$-generator called in the background.
If not provided, a descriptive exception is raised, for instance : 

In [7]:
try:
    QDelta = genQDeltaCoeffs("BE")
except Exception as e:
    print(f"{e.__class__.__name__}: {e}")

TypeError: TimeStepping.__init__() missing 1 required positional argument: 'nodes'


> 🔔 As for $Q$-generators, different aliases exists for each $Q_\Delta$-generators. For instance :

In [8]:
QDelta = genQDeltaCoeffs("IE", nodes=coll.nodes)  # equivalent to BE

And also, all those aliases are uniques among $Q_\Delta$-generators (so if the required alias is not found ... blablabla ...).
You can see all the current aliases and associated generators looking at the `QDELTA_GENERATORS` dictionary :

In [9]:
from qmat import QDELTA_GENERATORS

for i, (key, val) in enumerate(QDELTA_GENERATORS.items()):
    print(f"{key}: {val}")
    if i == 10:
        print("...")
        break   # only showing the first 10 aliases here ...

PIC: <class 'qmat.qdelta.algebraic.PIC'>
Picard: <class 'qmat.qdelta.algebraic.PIC'>
Exact: <class 'qmat.qdelta.algebraic.Exact'>
EXACT: <class 'qmat.qdelta.algebraic.Exact'>
LU: <class 'qmat.qdelta.algebraic.LU'>
LU2: <class 'qmat.qdelta.algebraic.LU2'>
QPar: <class 'qmat.qdelta.algebraic.QPar'>
Qpar: <class 'qmat.qdelta.algebraic.QPar'>
Qdiag: <class 'qmat.qdelta.algebraic.QPar'>
GS: <class 'qmat.qdelta.algebraic.GS'>
GaussSeidel: <class 'qmat.qdelta.algebraic.GS'>
...


## Use generator objects

As for $Q$-generators, you can also retrieve the $Q_\Delta$-generators with their classes, using one of the following approaches :

- import the generator directly from its submodule

In [10]:
from qmat.qdelta.timestepping import BE
approx = BE(nodes=coll.nodes)

- retrieve it with one of its aliases from the `QDELTA_GENERATORS` dictionary

In [11]:
Generator = QDELTA_GENERATORS["IE"]
approx = Generator(nodes=coll.nodes)

In both case, you'll instantiate an object that provides $Q_\Delta$ matrix through a `getQDelta` method :

In [12]:
print(approx.getQDelta())

[[0.08858796 0.         0.         0.        ]
 [0.08858796 0.3208789  0.         0.        ]
 [0.08858796 0.3208789  0.3781926  0.        ]
 [0.08858796 0.3208789  0.3781926  0.21234054]]


> ⚠️ While the $Q_\Delta$ generators do have a `QDelta` attribute, it is usually zero-initialized before calling the `getQDelta` method !

The reason behind the use of a method here (and not a property) is that some approximations can vary depending on a given iteration number, _e.g_ :

In [13]:
approx = QDELTA_GENERATORS["MIN-SR-FLEX"](nNodes=coll.nNodes, nodeType=coll.nodeType, quadType=coll.quadType)

print("Approximation for k=1 :")
print(approx.getQDelta(k=1))
print("Approximation for k=2 :")
print(approx.getQDelta(k=2))

Approximation for k=1 :
[[0.08858796 0.         0.         0.        ]
 [0.         0.40946686 0.         0.        ]
 [0.         0.         0.78765946 0.        ]
 [0.         0.         0.         1.        ]]
Approximation for k=2 :
[[0.04429398 0.         0.         0.        ]
 [0.         0.20473343 0.         0.        ]
 [0.         0.         0.39382973 0.        ]
 [0.         0.         0.         0.5       ]]


However this corresponds to some very specific methods, in practice most of the time any $k$ argument given to the `getQDelta` method will be ignored.

Finally, note that all $Q_\Delta$-generator objects have a `genCoeffs` method, which per default provides the same result as the `getQDelta` method, but has some more functionalities (covered in a later tutorial).

Next tutorial : [build a Spectral Deferred Correction type time-stepper ...](./04_sdc.ipynb)